In [0]:
# Given below #Jira ticket data and holiday data:
# | ticket_id | create_date | resolved_date |
# | 1 | 2022-08-01 | 2022-08-03 |
# | 2 | 2022-08-01 | 2022-08-12 |
# | 3 | 2022-08-01 | 2022-08-16 |

# | holiday_date |
# | 2022-08-11 |
# | 2022-08-15 |

# ❓Find out the total working days between ticket creation and resolution dates.

# Notes: Exclude weekend i.e saturday and sunday

In [0]:
from datetime import datetime

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DateType

In [0]:
my_schema=StructType(
    [
        StructField("ticket_id",IntegerType(),True),
        StructField("create_date",DateType(),True),
        StructField("resolved_date",DateType(),True)
    ]
)

In [0]:
mydata=[(1,datetime.strptime('2022-08-01','%Y-%m-%d'),datetime.strptime('2022-08-03','%Y-%m-%d')),
        (2,datetime.strptime('2022-08-01','%Y-%m-%d'),datetime.strptime('2022-08-12','%Y-%m-%d')),
        (3,datetime.strptime('2022-08-01','%Y-%m-%d'),datetime.strptime('2022-08-16','%Y-%m-%d'))]

In [0]:
tickets=spark.createDataFrame(data=mydata,schema=my_schema)

In [0]:
display(tickets)

ticket_id,create_date,resolved_date
1,2022-08-01,2022-08-03
2,2022-08-01,2022-08-12
3,2022-08-01,2022-08-16


In [0]:
holiday_schema=StructType([
    StructField("holiday_date",DateType(),True),
    StructField("reason",StringType(),True)
])

In [0]:
holiday_data=[(datetime.strptime('2022-08-11','%Y-%m-%d'),"Rakhi"),(datetime.strptime('2022-08-15','%Y-%m-%d'),"Independence Day")]

In [0]:
holidays=spark.createDataFrame(data=holiday_data,schema=holiday_schema)

In [0]:
display(holidays)

holiday_date,reason
2022-08-11,Rakhi
2022-08-15,Independence Day


In [0]:
display(tickets)

ticket_id,create_date,resolved_date
1,2022-08-01,2022-08-03
2,2022-08-01,2022-08-12
3,2022-08-01,2022-08-16


In [0]:
from pyspark.sql.functions import datediff,date_part,lit,floor,col,count

In [0]:
#Between one week difference there will be two weekends so we are doing minus with 2*difference in days
tickets.select(tickets.create_date,
               tickets.resolved_date,
               datediff(tickets.resolved_date,tickets.create_date).alias("actual_days"),
               date_part(lit('week'),tickets.create_date).alias("create_date_week"),
               date_part(lit('week'),tickets.resolved_date).alias("resolved_date_week"),
               floor((datediff(tickets.resolved_date,tickets.create_date)/7)).alias("week_difference"),
               (datediff(tickets.resolved_date,tickets.create_date) - 2*(floor((datediff(tickets.resolved_date,tickets.create_date)/7)))).alias("excluded_weekend_days")).show()

+-----------+-------------+-----------+----------------+------------------+---------------+---------------------+
|create_date|resolved_date|actual_days|create_date_week|resolved_date_week|week_difference|excluded_weekend_days|
+-----------+-------------+-----------+----------------+------------------+---------------+---------------------+
| 2022-08-01|   2022-08-03|          2|              31|                31|              0|                    2|
| 2022-08-01|   2022-08-12|         11|              31|                32|              1|                    9|
| 2022-08-01|   2022-08-16|         15|              31|                33|              2|                   11|
+-----------+-------------+-----------+----------------+------------------+---------------+---------------------+



In [0]:
from pyspark.sql.window import Window

In [0]:
joined_data=tickets.join(holidays,(tickets.create_date < holidays.holiday_date) & (holidays.holiday_date < tickets.resolved_date),"LEFT")
#joined_data.withColumn("count_holiday_day",count("holiday_date").over(Window.partitionBy(*["ticket_id","create_date","resolved_date"]))).select(col("count_holiday_day")).show()
#joined_data.select(count("holiday_date").over(Window.partitionBy(*["ticket_id","create_date","resolved_date"])).alias("count_holiday_day")).show()
joined_data=joined_data.groupBy("ticket_id","create_date","resolved_date").agg(count('holiday_date').alias("count_holiday_day"))

In [0]:
display(joined_data)

ticket_id,create_date,resolved_date,count_holiday_day
1,2022-08-01,2022-08-03,0
2,2022-08-01,2022-08-12,1
3,2022-08-01,2022-08-16,2


In [0]:
joined_data.select(joined_data.ticket_id,
                   joined_data.create_date,
               joined_data.resolved_date,
               datediff(joined_data.resolved_date,joined_data.create_date).alias("actual_days"),
               ((datediff(tickets.resolved_date,tickets.create_date) - 2*(floor((datediff(tickets.resolved_date,tickets.create_date)/7)))) - col("count_holiday_day")).alias("actual_ticket_close_day")).show()

+---------+-----------+-------------+-----------+-----------------------+
|ticket_id|create_date|resolved_date|actual_days|actual_ticket_close_day|
+---------+-----------+-------------+-----------+-----------------------+
|        1| 2022-08-01|   2022-08-03|          2|                      2|
|        2| 2022-08-01|   2022-08-12|         11|                      8|
|        3| 2022-08-01|   2022-08-16|         15|                      9|
+---------+-----------+-------------+-----------+-----------------------+

